**Comparing Psilocybin, LSD & Ketamine Neural PCA results using the Participation Ratio**

1. Read in dataframes

2. Participation Ratio Function

3. For Placebo (N=40) select all possible combinations of 22 (so 113380261800), and then randomly subsample 100 to calculate ED and build distribution

4. For Ketamine (N=40) select all possible combinations of 22 (so 113380261800), and then randomly subsample 100 to calculate ED and build distribution

5. For LSD (N=24) select all possible combinations of 22 (so 276), calculate ED on all and build distribution

6. For Psilocybin (N=23) jacknife and build a distribution of the PR values (22 values)

7. Plot the distributions

8. Run ANOVA

9. post hoc t-tests

10. Select 23 low motion ketamine participants, as with psilocybin jacknife and build a distribution of the PR values (22 values) and compare to lsd & psilo?

In [2]:
import pandas as pd
import numpy as np
from astropy.stats import jackknife_resampling
from astropy.stats import jackknife_stats
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import resample

import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp as mc
from scipy import stats

In [5]:
# 1. Read in dataframes

delta_ket = pd.read_csv('GBC_ket-pla_data.txt', header = None, delimiter = "\t")
pla_ket = pd.read_csv('GBC_pla_data_ketstudy.txt', header = None, delimiter = "\t")
ket = pd.read_csv('GBC_ket_data.txt', header = None, delimiter = "\t")

delta_psi = pd.read_csv('GBC_psi-pla_data.txt', header = None, delimiter = "\t")
pla_psi = pd.read_csv('GBC_pla_data_psilostudy.txt', header = None, delimiter = "\t")
psi = pd.read_csv('GBC_psi_data.txt', header = None, delimiter = "\t")

delta_lsd = pd.read_csv('GBC_lsd-pla_data.txt', header = None, delimiter = "\t")
pla_lsd = pd.read_csv('GBC_pla_data_lsdstudy.txt', header = None, delimiter = "\t")
lsd = pd.read_csv('GBC_lsd_data.txt', header = None, delimiter = "\t")

# test shapes
print('shape : ', lsd.shape)


shape :  (718, 24)


**Participation Ratio**

The PR Dimensionality of a set of points measures how many axes are needed to capture most of the variance of the points. The quantity is derived from the eigenvalues of the covariance matrix of the points. The quantity is maximized when all eigenvalues are equal, and minimized when one eigenvalue is much larger than the rest.

(Farrell, M., Recanatesi, S., Lajoie, G., & Shea-Brown, E. (2019). Dynamic compression and expansion in a classifying recurrent network. bioRxiv, 564476.)

In [23]:
# 2. this function returns the participation ratio and the eigenvalues for an arbitrary data array

def participation(data):
    cov = np.cov(data)
    evls = np.linalg.eig(cov)[0].real
    return 1/np.sum((evls/np.sum(evls))**2),evls

# this one only returns the PR
def participation_ratio_only(data):
    cov = np.cov(data)
    evls = np.linalg.eig(cov)[0].real
    return 1/np.sum((evls/np.sum(evls))**2)

In [24]:
# double check what shape the covariance matrixes will take that will be input into the participant 
# ratio calculation (should be 718 x 718 which was also input into the pca):

pla_ket_cov = np.cov(pla_ket)
print('pla covariance matrix shape : ', pla_ket_cov.shape)

NameError: name 'pla_ket' is not defined

In [15]:
# participation ratio with no subsampling/jackknifing 
print('Ketamine (N=40):')
print('Placebo PR: ', participation_ratio_only(pla_ket))
print('Ketamine PR: ', participation_ratio_only(ket))
print('Delta (Ket - Pla) PR: ', participation_ratio_only(delta_ket))

print('Psilocybin (N=23):')
print('Placebo PR: ', participation_ratio_only(pla_psi))
print('Psilocybin PR: ', participation_ratio_only(psi))
print('Delta (Psi - Pla) PR: ', participation_ratio_only(delta_psi))

print('LSD (N=24):')
print('Placebo PR: ', participation_ratio_only(pla_lsd))
print('LSD PR: ', participation_ratio_only(lsd))
print('Delta (LSD - Pla) PR: ', participation_ratio_only(delta_lsd))


Ketamine (N=40):
Placebo PR:  14.197129406719224
Ketamine PR:  17.99701485778362
Delta (Ket - Pla) PR:  18.34524656134304
Psilocybin (N=23):
Placebo PR:  6.275811322320774
Psilocybin PR:  7.688264201159916
Delta (Psi - Pla) PR:  8.86659328115698
LSD (N=24):
Placebo PR:  8.705244510466159
LSD PR:  5.962443127700551
Delta (LSD - Pla) PR:  9.106029172731791


In [16]:
# 3. For Placebo (N=40) 

# create a array of 100 as that is how many times we want to subsample & calculate PR
numb = np.array(list(range(0, 100)))

# create an array from 1 - 40 as these are the subjects we have to subsample from
data = np.array(list(range(0, 40)))

# create empty array to add the PR values to
pla_ket_prs = []

for i in numb:
    # create the array of subjects to sample from changing the random_state each loop
    samples = resample(data, replace=False, n_samples=22, random_state=i)
    # use the array of samples to select the ketamine subjects
    pla_ket_22 = pla_ket.loc[:,samples]
    # compute PR
    pla_ket_prs.append(participation_ratio_only(pla_ket_22))

# subsample estimate:
print('Pla subsampling estimate: ', np.mean(pla_ket_prs))

# save pr values as dataframe with column PR
pla_ket_prs_df = pd.DataFrame(pla_ket_prs, columns = ['PR'])

# double check shape of covariance matrix
print('pla_ket covariance matrix shape : ', pla_ket_22.shape)


Pla subsampling estimate:  10.404243582215583
pla_ket covariance matrix shape :  (718, 22)


**Pla subsampling estimate:  10.404243582215583 <br>
pla_ket covariance matrix shape :  (718, 22)**

In [18]:
# 4. For Ket (N=40) 

# create a array of 100 as that is how many times we want to subsample & calculate PR
numb = np.array(list(range(0, 100)))

# create an array from 1 - 40 as these are the subjects we have to subsample from
data = np.array(list(range(0, 40)))

# create empty array to add the PR values to
ket_prs = []

for i in numb:
    # create the array of subjects to sample from changing the random_state each loop
    samples = resample(data, replace=False, n_samples=22, random_state=i)
    # use the array of samples to select the ketamine subjects
    ket_22 = ket.loc[:,samples]
    # compute PR
    ket_prs.append(participation_ratio_only(ket_22))

# subsample estimate:
print('Ket subsampling estimate: ', np.mean(ket_prs))

# save pr values as dataframe with column PR
ket_prs_df = pd.DataFrame(ket_prs, columns = ['PR'])

# double check shape of covariance matrix
print('pla covariance matrix shape : ', ket_22.shape)

Ket subsampling estimate:  12.038611155192477
pla covariance matrix shape :  (718, 22)


**Ket subsampling estimate:  12.038611155192477 <br>
pla covariance matrix shape :  (718, 22)**

In [17]:
# 5. For Delta (N=40) 

# create a array of 100 as that is how many times we want to subsample & calculate PR
numb = np.array(list(range(0, 100)))

# create an array from 1 - 40 as these are the subjects we have to subsample from
data = np.array(list(range(0, 40)))

# create empty array to add the PR values to
delta_ket_prs = []

for i in numb:
    # create the array of subjects to sample from changing the random_state each loop
    samples = resample(data, replace=False, n_samples=22, random_state=i)
    # use the array of samples to select the ketamine subjects
    delta_ket_22 = delta_ket.loc[:,samples]
    # compute PR
    delta_ket_prs.append(participation_ratio_only(delta_ket_22))

# subsample estimate:
print('delta_ket subsampling estimate: ', np.mean(delta_ket_prs))

# save pr values as dataframe with column PR
delta_ket_prs_df = pd.DataFrame(delta_ket_prs, columns = ['PR'])

# double check shape of covariance matrix
print('delta_ket covariance matrix shape : ', delta_ket_22.shape)


delta_ket subsampling estimate:  12.301844204267953
delta_ket covariance matrix shape :  (718, 22)


**delta_ket subsampling estimate:  12.301844204267953 <br>
delta_ket covariance matrix shape :  (718, 22)**

In [8]:
# 5. For LSD all (N=24) select all possible combinations of 22 (so 276), calculate ED on all and build distribution

In [19]:
# function to Return r length subsequences of elements from the input iterable.

# The combination tuples are emitted in lexicographic ordering according to the order of the input iterable. 
# So, if the input iterable is sorted, the combination tuples will be produced in sorted order.

# Elements are treated as unique based on their position, not on their value. 
# So if the input elements are unique, there will be no repeat values in each combination.

def combinations(iterable, r):
    # combinations('ABCD', 2) --> AB AC AD BC BD CD
    # combinations(range(4), 3) --> 012 013 023 123
    pool = tuple(iterable)
    n = len(pool)
    if r > n:
        return
    indices = list(range(r))
    yield tuple(pool[i] for i in indices)
    while True:
        for i in reversed(range(r)):
            if indices[i] != i + n - r:
                break
        else:
            return
        indices[i] += 1
        for j in range(i+1, r):
            indices[j] = indices[j-1] + 1
        yield tuple(pool[i] for i in indices)

In [20]:
# create an array for 24 values as these are the subjects we have to subsample from
data = np.array(list(range(0, 24)))

In [21]:
# find all possible combinations of 22 participants from the 24

comb = combinations(data, 22)
comb_df = pd.DataFrame(comb)
print('Number of possible combinations for LSD: ', comb_df.shape)

# should get 276 combinations

Number of possible combinations for LSD:  (276, 22)


In [22]:
# PLACEBO LSD

# create a array of 276 values (as will be computing PR 276 times)
numb = np.array(list(range(0, 276)))

# create empty array to add the PR values to
pla_lsd_prs = []

for i in numb:
    # select the combination from comb that corresponds to the loop value i and save as p
    p = comb_df.iloc[i]
    # use p to select the participants you want from the lsd dataframe
    pla_lsd_22 = pla_lsd.loc[:,p]
    # compute PR abd append to lsd_prs
    pla_lsd_prs.append(participation_ratio_only(pla_lsd_22))

# estimate:
print('Pla (LSD) estimate: ', np.mean(pla_lsd_prs))

# save pr values as dataframe with column PR
pla_lsd_prs_df = pd.DataFrame(pla_lsd_prs, columns = ['PR'])

# double check shape of covariance matrix
print('pla (lsd) covariance matrix shape : ', pla_lsd_22.shape)

# save dataframe to csv as takes a while to calculate
pla_lsd_prs_df.to_csv(r'pla_lsd_prs_df.csv')

Pla (LSD) estimate:  8.381841540701009
pla (lsd) covariance matrix shape :  (718, 22)


**Pla (LSD) estimate:  8.381841540701009 <br>
pla (lsd) covariance matrix shape :  (718, 22)**

In [23]:
# LSD

# loop to get PR for each combinations:

# create a array of 276 values (as will be computing PR 276 times)
numb = np.array(list(range(0, 276)))

# create empty array to add the PR values to
lsd_prs = []

for i in numb:
    # select the combination from comb that corresponds to the loop value i and save as p
    p = comb_df.iloc[i]
    # use p to select the participants you want from the lsd dataframe
    lsd_22 = lsd.loc[:,p]
    # compute PR abd append to lsd_prs
    lsd_prs.append(participation_ratio_only(lsd_22))

# estimate:
print('LSD estimate: ', np.mean(lsd_prs))

# save pr values as dataframe with column PR
lsd_prs_df = pd.DataFrame(lsd_prs, columns = ['PR'])

# double check shape of covariance matrix
print('lsd covariance matrix shape : ', lsd_22.shape)

LSD estimate:  5.755794305505647
lsd covariance matrix shape :  (718, 22)


**LSD estimate:  5.755794305505647 <br>
lsd covariance matrix shape :  (718, 22)**

In [26]:
# DELTA LSD

# loop to get PR for each combinations:

# create a array of 276 values (as will be computing PR 276 times)
numb = np.array(list(range(0, 276)))

# create empty array to add the PR values to
delta_lsd_prs = []

for i in numb:
    # select the combination from comb that corresponds to the loop value i and save as p
    p = comb_df.iloc[i]
    # use p to select the participants you want from the lsd dataframe
    delta_lsd_22 = delta_lsd.loc[:,p]
    # compute PR abd append to lsd_prs
    delta_lsd_prs.append(participation_ratio_only(delta_lsd_22))

# estimate:
print('Delta estimate: ', np.mean(delta_lsd_prs))

# save pr values as dataframe with column PR
delta_lsd_prs_df = pd.DataFrame(delta_lsd_prs, columns = ['PR'])

# double check shape of covariance matrix
print('delta_lsd covariance matrix shape : ', delta_lsd_22.shape)

# save dataframe to csv as takes a while to calculate
delta_lsd_prs_df.to_csv(r'delta_lsd_prs_df.csv')

Delta estimate:  8.691337384170165
delta_lsd covariance matrix shape :  (718, 22)


**Delta estimate:  8.691337384170165 <br>
delta_lsd covariance matrix shape :  (718, 22)**

In [27]:
# 6. For Placebo (Psilocybin) (N=23) jacknife and build a distribution of the PR values (22 values)

# create a array from 1 - 22 (as the total no. of values we want is 22)
numb = np.array(list(range(0, 22)))

# create empty array to add the PR values to
pla_psi_prs = []

for i in numb:
    temp = pla_psi
    # drop the subject
    temp = temp.drop(labels=i, axis=1)
    # compute PR and add to list
    pla_psi_prs.append(participation_ratio_only(temp))

# jackknife estimate:
print('pla_psi Jackknife estimate: ', np.mean(pla_psi_prs))

# save pr values as dataframe with column PR
pla_psi_prs_df = pd.DataFrame(pla_psi_prs, columns = ['PR'])

# double check shape of covariance matrix
print('pla_psi covariance matrix shape : ', temp.shape)

pla_psi Jackknife estimate:  6.162950402127771
pla_psi covariance matrix shape :  (718, 22)


**pla_psi Jackknife estimate:  6.162950402127771 <br>
pla_psi covariance matrix shape :  (718, 22)**

In [28]:
# 6. For Psilocybin (N=23) jacknife and build a distribution of the PR values (22 values)

# create a array from 1 - 22 (as the total no. of values we want is 22)
numb = np.array(list(range(0, 22)))

# create empty array to add the PR values to
psi_prs = []

for i in numb:
    temp = psi
    # drop the subject
    temp = temp.drop(labels=i, axis=1)
    # compute PR and add to list
    psi_prs.append(participation_ratio_only(temp))

# jackknife estimate:
print('Psi Jackknife estimate: ', np.mean(psi_prs))

# save pr values as dataframe with column PR
psi_prs_df = pd.DataFrame(psi_prs, columns = ['PR'])

# double check shape of covariance matrix
print('psi covariance matrix shape : ', temp.shape)

Psi Jackknife estimate:  7.5079528738228865
psi covariance matrix shape :  (718, 22)


**Psi Jackknife estimate:  7.5079528738228865 <br>
psi covariance matrix shape :  (718, 22)**

In [30]:
# 6. For Delta (Psilocybin - Placebo) (N=23) jacknife and build a distribution of the PR values (22 values)

# create a array from 1 - 22 (as the total no. of values we want is 22)
numb = np.array(list(range(0, 22)))

# create empty array to add the PR values to
delta_psi_prs = []

for i in numb:
    temp = delta_psi
    # drop the subject
    temp = temp.drop(labels=i, axis=1)
    # compute PR and add to list
    delta_psi_prs.append(participation_ratio_only(temp))

# jackknife estimate:
print('delta_psi Jackknife estimate: ', np.mean(delta_psi_prs))

# save pr values as dataframe with column PR
delta_psi_prs_df = pd.DataFrame(delta_psi_prs, columns = ['PR'])

# double check shape of covariance matrix
print('delta_psi covariance matrix shape : ', temp.shape)

delta_psi Jackknife estimate:  8.635334792178678
delta_psi covariance matrix shape :  (718, 22)


**delta_psi Jackknife estimate:  8.635334792178678 <br>
delta_psi covariance matrix shape :  (718, 22)**

In [34]:
# 7. Plot the distributions (save the dataset to run in R)

pla_ket_prs_df["condition"] = "pla_ket"
ket_prs_df["condition"] = "ket"
delta_ket_prs_df["condition"] = "delta_ket"

pla_psi_prs_df["condition"] = "pla_psi"
psi_prs_df["condition"] = "psi"
delta_psi_prs_df["condition"] = "delta_psi"

pla_lsd_prs_df["condition"] = "pla_lsd"
lsd_prs_df["condition"] = "lsd"
delta_lsd_prs_df["condition"] = "delta_lsd"

dfall = pla_ket_prs_df
dfall2 = dfall.append(ket_prs_df, ignore_index=True)
dfall3 = dfall2.append(delta_ket_prs_df, ignore_index=True)
dfall4 = dfall3.append(pla_psi_prs_df, ignore_index=True)
dfall5 = dfall4.append(psi_prs_df, ignore_index=True)
dfall6 = dfall5.append(delta_psi_prs_df, ignore_index=True)
dfall7 = dfall6.append(pla_lsd_prs_df, ignore_index=True)
dfall8 = dfall7.append(lsd_prs_df, ignore_index=True)
dfall9 = dfall8.append(delta_lsd_prs_df, ignore_index=True)

dfall9.to_csv(r'participationratio_19_7_21.csv')

In [35]:
# 8. Run ANOVA 

model = ols('PR ~ C(condition)', data=dfall9).fit()
aov_table = sm.stats.anova_lm(model, typ=2)
aov_table

,sum_sq,df,F,PR(>F)
C(condition),5283.362876,8.0,2089.292138,0.0
Residual,374.575729,1185.0,NaN,NaN


In [36]:
# 9. post hoc tests

comp = mc.MultiComparison(dfall9['PR'], dfall9['condition'])
tbl, a1, a2 = comp.allpairtest(stats.ttest_ind, method= "bonf")

tbl

group1,group2,stat,pval,pval_corr,reject
delta_ket,delta_lsd,75.212,0.0,0.0,True
delta_ket,delta_psi,25.5215,0.0,0.0,True
delta_ket,ket,2.6344,0.0091,0.3274,False
delta_ket,lsd,83.2188,0.0,0.0,True
delta_ket,pla_ket,16.6686,0.0,0.0,True
delta_ket,pla_lsd,74.1698,0.0,0.0,True
delta_ket,pla_psi,42.6637,0.0,0.0,True
delta_ket,psi,32.2318,0.0,0.0,True
delta_lsd,delta_psi,0.9212,0.3577,1.0,False
delta_lsd,ket,-63.4209,0.0,0.0,True


In [19]:
# 10.1 Select 23 low motion ketamine participants, as with psilocybin jacknife and build a distribution of 
# the PR values (22 values) and compare to lsd & psilo?


ket23 = pd.read_csv('GBC_ket-pla_data_23_low_motion.txt', header = None, delimiter = "\t")

# test shapes
print('shape : ', low_motion_ket.shape)

shape :  (718, 23)


In [27]:
# 10.2 For Delta (Ket - Pla) (N=23) jacknife and build a distribution of the PR values (22 values)

# create a array from 1 - 22 (as the total no. of values we want is 22)
numb = np.array(list(range(0, 22)))

# create empty array to add the PR values to
delta_ket23_prs = []

for i in numb:
    temp = ket23
    # drop the subject
    temp = temp.drop(labels=i, axis=1)
    # compute PR and add to list
    delta_ket23_prs.append(participation_ratio_only(temp))

# jackknife estimate:
print('delta_ket Jackknife estimate: ', np.mean(delta_ket23_prs))

# save pr values as dataframe with column PR
delta_ket23_prs_df = pd.DataFrame(delta_ket23_prs, columns = ['PR'])

# double check shape of covariance matrix
print('delta_ket23 covariance matrix shape : ', temp.shape)

delta_ket Jackknife estimate:  12.261622308675236
delta_ket23 covariance matrix shape :  (718, 22)


In [28]:
delta_ket23_prs_df


,PR
0,12.220314
1,12.316588
2,12.322477
3,12.454934
4,12.227168
5,11.905531
6,12.275117
7,12.221086
8,12.465308
9,12.260656


In [30]:
# 10.1 Select 23 high motion ketamine participants, as with psilocybin jacknife and build a distribution of 
# the PR values (22 values) and compare to lsd & psilo?


ket23_high = pd.read_csv('GBC_ket-pla_data_23_high_motion.txt', header = None, delimiter = "\t")

# test shapes
print('shape : ', ket23_high.shape)

# 10.2 For Delta (Ket - Pla) (N=23) jacknife and build a distribution of the PR values (22 values)

# create a array from 1 - 22 (as the total no. of values we want is 22)
numb = np.array(list(range(0, 22)))

# create empty array to add the PR values to
delta_ket23_high_prs = []

for i in numb:
    temp = ket23_high
    # drop the subject
    temp = temp.drop(labels=i, axis=1)
    # compute PR and add to list
    delta_ket23_high_prs.append(participation_ratio_only(temp))

# jackknife estimate:
print('delta_ket Jackknife estimate: ', np.mean(delta_ket23_high_prs))

# save pr values as dataframe with column PR
delta_ket23_high_prs_df = pd.DataFrame(delta_ket23_high_prs, columns = ['PR'])

# double check shape of covariance matrix
print('delta_ket23 covariance matrix shape : ', temp.shape)

delta_ket23_high_prs_df

shape :  (718, 23)
delta_ket Jackknife estimate:  12.648016900001908
delta_ket23 covariance matrix shape :  (718, 22)


,PR
0,12.741466
1,12.306904
2,12.710079
3,12.647613
4,12.630321
5,12.622661
6,12.656154
7,12.434469
8,12.624409
9,12.859813


In [34]:
# 10.4 ket substance
# high ket substance: GBC_ket_substance_high23_data.txt
# low ket substance: GBC_ket_substance_low23_data.txt
# high ket study placebo: GBC_pla_data_ketstudy_high23.txt
# low ket study placebo: GBC_pla_data_ketstudy_low23.txt

ket23 = pd.read_csv('GBC_pla_data_ketstudy_low23.txt', header = None, delimiter = "\t")

# test shapes
print('shape : ', ket23.shape)

# 10.2 For Delta (Ket - Pla) (N=23) jacknife and build a distribution of the PR values (22 values)

# create a array from 1 - 22 (as the total no. of values we want is 22)
numb = np.array(list(range(0, 22)))

# create empty array to add the PR values to
delta_ket23_prs = []

for i in numb:
    temp = ket23
    # drop the subject
    temp = temp.drop(labels=i, axis=1)
    # compute PR and add to list
    delta_ket23_prs.append(participation_ratio_only(temp))

# jackknife estimate:
print('delta_ket Jackknife estimate: ', np.mean(delta_ket23_prs))

# save pr values as dataframe with column PR
delta_ket23_prs_df = pd.DataFrame(delta_ket23_prs, columns = ['PR'])

# double check shape of covariance matrix
print('delta_ket23 covariance matrix shape : ', temp.shape)

delta_ket23_prs_df

shape :  (718, 23)
delta_ket Jackknife estimate:  11.379684673315415
delta_ket23 covariance matrix shape :  (718, 22)


,PR
0,12.074805
1,11.451604
2,11.197702
3,11.174368
4,11.648143
5,11.036606
6,11.346303
7,11.885298
8,10.981222
9,11.004985
